In [12]:
from keras.applications import EfficientNetB0
from keras.applications.efficientnet import preprocess_input
from keras.preprocessing.image import load_img, img_to_array
from keras.models import Model
import numpy as np
import os


In [ ]:

# Load base model without classification head
base_model = EfficientNetB0(include_top=False, weights='imagenet', pooling='avg')


Idea: our algorithm will give us pure-noise points. Try sampling x and y from the training data and we'll get an image which is a linear interpo of x and y.

The point of this notebook is to implement a simple line-search method for finding points
where the Lipschitz ratio is large. 

Define the Lipschitz ratio $k(x, y) = \frac{||f(x) - f(y)||}{||x-y||}$. This measures how large a change happens in the output (eg, in an output embedding, or at the logit layer) for a given amount of change in the input.

Then the Lipschitz constant $K = \max_{x, y \in X} k(x, y)$, or if we are mathematicians, $\sup$ instead of $\max$.

The line-search method is based on the observation that if $k(x, y)$ is large, then we can split the line segment $[x, y]$ by choosing the midpoint $z = (x + y) / 2$, and either $k(x, z) \ge k(x, y)$, or $k(z, y) \ge k(x, y)$, or even both. This follows from the triangle inequality in the output space, or the mean-value theorem. Therefore we can pick the half-segment that gives the larger $k$, "zoom in", and iterate. This will give a non-decreasing sequence of $k$ values. We can terminate when the interval length is comparable to floating-point minimality. We will have found a pair of points very close by with very large $k$. 

By this algorithm we have proved that if $K$ is actually achieved (max, not sup), then it is actually achieved by two points arbitrarily nearby each other.

Of course, $k$ is analogous to the definition of the derivative $\lim_{h\rightarrow 0} \frac{f(x+h) - f(x)}{h}$ *before* letting $h \rightarrow 0$, and $K$ *after*.

In practice, on EfficientNet, I am seeing that initial randomly-sampled points gives $k = 0.001$, and at the end of the line-search we have $k = 0.1$.

Below, lots of code generated by Claude. I believe all is correct. 

But some is more complex than it needs to be. Some functions try to handle both (224, 224, 3) and (b, 224, 224, 3)
and we would be better without this. 

It would be good to fully vectorise the algorithm, maybe.

Need to check that preprocessing is correct always.

Check that random images are in the right space, ie the right range.

About floating-point error: it could be misleading when the two points are very close together, if many of the elements of the outputs land "in between" available FP values and all pairs happen to be rounded away from each other. That would give a misleadingly large value for $||f(x) - f(y)||$.

I have some images for this notebook.



In [13]:

def get_embedding(x):
    """
    Convert image(s) to embeddings using EfficientNet.
    Expects x to already be preprocessed (in range [-1, 1]).
    """
    return base_model.predict(x, verbose=0)

def load_image_batch(image_path, target_size=(224, 224)):
    """
    Load and preprocess a single image.
    Returns preprocessed image in range [-1, 1].
    """
    img = load_img(image_path, target_size=target_size)
    x = img_to_array(img)  # Returns values in [0, 255]
    x = np.expand_dims(x, axis=0)
    return preprocess_input(x)  # Scales to [-1, 1] and applies ImageNet preprocessing

def load_image_dataset(directory, num_images=100):
    """Load multiple images from a directory"""
    images = []
    image_paths = []
    
    for root, _, files in os.walk(directory):
        for filename in files:
            if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
                image_paths.append(os.path.join(root, filename))
                if len(image_paths) >= num_images:
                    break
        if len(image_paths) >= num_images:
            break
    
    for path in image_paths:
        try:
            img = load_image_batch(path)
            images.append(img)
        except Exception as e:
            print(f"Error loading {path}: {e}")
            continue
    
    return np.vstack(images) if images else np.array([])

def generate_random_images(num_images=100):
    """
    Generate random images for testing.
    Returns preprocessed images in range [-1, 1].
    """
    # Generate in [0, 255] first
    raw_images = np.random.randint(0, 256, (num_images, 224, 224, 3)).astype(np.float32)
    # Then preprocess to [-1, 1]
    return preprocess_input(raw_images)


# Example usage:
"""
# Replace 'path_to_images' with your image directory
images = load_image_dataset('path_to_images', num_images=100)
embeddings = get_embedding(images)

print("Images shape:", images.shape)
print("Embeddings shape:", embeddings.shape)
"""


'\n# Replace \'path_to_images\' with your image directory\nimages = load_image_dataset(\'path_to_images\', num_images=100)\nembeddings = get_embedding(images)\n\nprint("Images shape:", images.shape)\nprint("Embeddings shape:", embeddings.shape)\n'

In [5]:
# Generate some random test images
test_images = generate_random_images(100)

# Generate and check random images
test_images = generate_random_images(5)
print("\nRandom image value ranges:")
print("Min value:", test_images.min())
print("Max value:", test_images.max())

# Get and check embeddings
test_embeddings = get_embedding(test_images)
print("\nEmbedding shapes:")
print("Test images shape:", test_images.shape)
print("Test embeddings shape:", test_embeddings.shape)
print("Embedding dimension:", test_embeddings.shape[1])


Random image value ranges:
Min value: 1.3153993672077036e-07
Max value: 0.9999986656611144

Embedding shapes:
Test images shape: (5, 224, 224, 3)
Test embeddings shape: (5, 1280)
Embedding dimension: 1280


In [20]:
import numpy as np

def input_distance(x1, x2):
    """
    Compute Frobenius (Euclidean) norm between two input images.
    
    Args:
        x1, x2: numpy arrays of shape (224, 224, 3) or batches (b, 224, 224, 3)
        
    Returns:
        Float or array of distances if inputs are batches
    """
    # Reshape inputs to 2D if needed
    if x1.ndim > 2:
        shape1 = x1.shape
        shape2 = x2.shape
        x1 = x1.reshape(shape1[0], -1) if x1.ndim == 4 else x1.reshape(1, -1)
        x2 = x2.reshape(shape2[0], -1) if x2.ndim == 4 else x2.reshape(1, -1)
    
    # Compute Euclidean distance
    return np.sqrt(np.sum((x1 - x2) ** 2, axis=1))

def output_distance(f1, f2):
    """
    Compute Frobenius (Euclidean) norm between two embedding vectors.
    
    Args:
        f1, f2: numpy arrays of shape (embedding_dim,) or batches (b, embedding_dim)
        
    Returns:
        Float or array of distances if inputs are batches
    """
    # Add batch dimension if needed
    if f1.ndim == 1:
        f1 = f1.reshape(1, -1)
        f2 = f2.reshape(1, -1)
    
    # Compute Euclidean distance
    return np.sqrt(np.sum((f1 - f2) ** 2, axis=1))

def lipschitz_ratio(x1, x2, f1=None, f2=None):
    """
    Compute the ratio ||f(x1) - f(x2)|| / ||x1 - x2||
    
    Args:
        x1, x2: input images
        f1, f2: corresponding embeddings
        
    Returns:
        Float or array of ratios if inputs are batches
    """
    input_dist = input_distance(x1, x2)

    if f1 is None:
        f1 = get_embedding(x1)
    if f2 is None:
        f2 = get_embedding(x2)
    output_dist = output_distance(f1, f2)
    
    # Avoid division by zero
    mask = input_dist > 1e-10
    ratios = np.zeros_like(input_dist)
    ratios[mask] = output_dist[mask] / input_dist[mask]
    
    return ratios


# Generate some random test data
x1 = np.random.random((2, 224, 224, 3))
x2 = np.random.random((2, 224, 224, 3))
f1 = get_embedding(x1)
f2 = get_embedding(x2)

# Test distances
print("Input distances:", input_distance(x1, x2))
print("Output distances:", output_distance(f1, f2))
print("Lipschitz ratios:", lipschitz_ratio(x1, x2, f1, f2))

# Test single example
print("\nSingle example:")
print("Input distance:", input_distance(x1[0], x2[0]))
print("Output distance:", output_distance(f1[0], f2[0]))
print("Lipschitz ratio:", lipschitz_ratio(x1[0:1], x2[0:1], f1[0:1], f2[0:1]))
print("Lipschitz_ratio from x only:", lipschitz_ratio(x1[0:1], x2[0:1]))

Input distances: [158.10162281 158.21117368]
Output distances: [0.01115054 0.01903192]
Lipschitz ratios: [7.05276867e-05 1.20294420e-04]

Single example:
Input distance: [158.10162281]
Output distance: [0.01115054]
Lipschitz ratio: [7.05276867e-05]
Lipschitz_ratio from x only: [7.05242819e-05]


In [26]:
n = 100
X = generate_random_images(n)
Y = generate_random_images(n)
fX = get_embedding(X)
fY = get_embedding(Y)


In [27]:
ratios = lipschitz_ratio(X, Y, fX, fY)
max_ratio_idx = np.argmax(ratios)

In [28]:
max_ratio_idx

60

In [30]:
lipschitz_ratio(X[60:61], Y[60:61])

array([0.00017107], dtype=float32)

In [31]:
max(ratios)

0.00017107201

In [45]:
def lipschitz_line_search(x, y):
    epsilon = 0.00005 # smaller than this we can get "stuck" as z = (x+y)/2 could equal x or y in each dimension, due to floating-point error
    d_xy = input_distance(x, y)
    i = 0
    while d_xy > epsilon:
        z = (x + y) / 2
        fx = get_embedding(x)
        fy = get_embedding(y)
        fz = get_embedding(z)
        l_xz = lipschitz_ratio(x, z, fx, fz)
        l_zy = lipschitz_ratio(z, y, fz, fy)
        if l_xz > l_zy:
            y = z
            fy = fz
        else:
            x = z
            fx = fz
        new_d_xy = input_distance(x, y)
        if new_d_xy == d_xy:
            break
        d_xy = new_d_xy
        l_xy = lipschitz_ratio(x, y, fx, fy)
        print(i, d_xy, l_xy)
        i += 1
    return x, y

for i in range(19, 100):
    print(i)
    print(lipschitz_ratio(X[i:i+1], Y[i:i+1]))
    x, y = lipschitz_line_search(X[i:i+1], Y[i:i+1])
    print(lipschitz_ratio(x, y))

19
[5.566533e-05]
0 [20229.934] [0.00018036]
1 [10114.967] [0.00033383]
2 [5057.4834] [0.00063807]
3 [2528.7417] [0.0011291]
4 [1264.3708] [0.00161012]
5 [632.1854] [0.0018618]
6 [316.0927] [0.0019467]
7 [158.04636] [0.00197185]
8 [79.02318] [0.0019798]
9 [39.51159] [0.00198311]
10 [19.755795] [0.00198365]
11 [9.877897] [0.0019869]
12 [4.9389486] [0.00198873]
13 [2.4694743] [0.00199555]
14 [1.2347372] [0.0019963]
15 [0.6173686] [0.00200664]
16 [0.3086841] [0.0020157]
17 [0.15435793] [0.00216903]
18 [0.07720014] [0.00241065]
19 [0.03864492] [0.00245268]
20 [0.01939901] [0.00255029]
21 [0.00979492] [0.00364908]
22 [0.00515474] [0.0045941]
23 [0.00317484] [0.00535168]
24 [0.00260982] [0.00518637]
25 [0.00046154] [0.00911535]
26 [0.00024459] [0.01604819]
27 [8.907245e-05] [0.06027395]
28 [3.635556e-05] [0.12332452]
[0.12332452]
20
[7.765391e-05]
0 [20257.748] [0.00025984]
1 [10128.874] [0.0003977]
2 [5064.437] [0.00062053]
3 [2532.2185] [0.00100997]
4 [1266.1093] [0.00141371]
5 [633.0546] 